## Creating a sub-network of assembled interactions from a list of entities for Boolean network modeling

### Getting identifiers for a list of entities

In the spreadsheet provided below, informal names are given for several genes, proteins, and phenotypes. Unless database identifiers are readily available for these, we need to "ground" them automatically. The Gilda package (see http://grounding.indra.bio/, https://github.com/indralab/gilda, https://www.biorxiv.org/content/10.1101/2021.09.10.459803v1) can correctly ground most entities in the provided spreadsheet but some use non-standard naming like underscores so these we assign identifiers manually

In [1]:
fname = 'test_INDRA_BN_seed_genes_apoptosis.xlsx'

In [2]:
import gilda
import pandas

mappings = {
    'SARS_3a': ('UP', 'P59632'),
    'SARS_E7a': ('UP', 'P59637'),
    'SARS_M': ('UP', 'P59596'),
    'Procasp9': ('UP', 'P55211'),
    'Apoptosis_phenotype': ('GO', 'GO:0006915'),
    'Cytochrome_C': ('HGNC', '19986')
}


df = pandas.read_excel(fname, engine='openpyxl', header=None)
entities = sorted(set(df[0]))
groundings = {}
for entity in entities:
    mapped_grounding = mappings.get(entity)
    if mapped_grounding:
        groundings[entity] = mapped_grounding
    else:
        matches = gilda.ground(entity)
        if not matches:
            groundings[entity] = None
        else:
            groundings[entity] = matches[0].term.db, matches[0].term.id

At this point we have a dict called `groundings` that contains each entity's name as a key and its database namespace and identifier as value.

In [3]:
groundings

{'AKT': ('FPLX', 'AKT'),
 'Apoptosis_phenotype': ('GO', 'GO:0006915'),
 'Apoptosome': ('GO', 'GO:0043293'),
 'BAD': ('HGNC', '936'),
 'BAX': ('HGNC', '959'),
 'BCL2': ('HGNC', '990'),
 'BCLXL': ('HGNC', '992'),
 'BID': ('HGNC', '1050'),
 'BIM': ('HGNC', '994'),
 'CASP3': ('HGNC', '1504'),
 'CASP7': ('HGNC', '1508'),
 'CASP8': ('HGNC', '1509'),
 'CASP9': ('HGNC', '1511'),
 'Cytochrome_C': ('HGNC', '19986'),
 'FADD': ('HGNC', '3573'),
 'FAS': ('HGNC', '11920'),
 'FASL': ('HGNC', '11936'),
 'MAPK14': ('HGNC', '6876'),
 'MCL1': ('HGNC', '6943'),
 'PUMA': ('HGNC', '17868'),
 'Procasp9': ('UP', 'P55211'),
 'SARS_3a': ('UP', 'P59632'),
 'SARS_E7a': ('UP', 'P59637'),
 'SARS_M': ('UP', 'P59596'),
 'TNFA': ('HGNC', '11892'),
 'TNFR1': ('HGNC', '11916'),
 'TRADD': ('HGNC', '12030')}

### Obtaining the sub-network of relations from INDRA Discovery

With these in hand, we can now start constructing the subnetwork. To do this, we turn to the INDRA Discovery service (https://discovery.indra.bio) whose REST API is documented at https://discovery.indra.bio/apidocs. We call the `indra_subnetwork_relations` endpoint which takes a list of up to 300 database namespace/identifier pairs as input and returns all relations (each relation representing an INDRA Statement (see https://indra.readthedocs.io/en/latest/modules/statements.html)) between them. These relations have the following structure:
- `source_ns`: the source entity's database namespace
- `source_id`: the source entity's database identifier
- `source_ns`: the target entity's database namespace
- `source_id`: the target entity's database identifier
- `rel_type`: this is always `indra_rel` so not interesting
- `data`:
    - `stmt_hash`: the unique identifier of the INDRA Statement, derived from its content
    - `stmt_json`: the full data structure of the INDRA Statement including at most 1 of its evidences (even if the Statement is supported by many evidences, for efficiency, this endpoint doesn't return all of them. This JSON can be deserialized into INDRA Statement objects using the INDRA package if installed (see https://indra.readthedocs.io/en/latest/modules/statements.html#indra.statements.statements.stmts_from_json). There is also a JSON Schema specification for INDRA Statements here in case it's useful: https://github.com/sorgerlab/indra/blob/master/indra/resources/statements_schema.json
    - `evidence_count`: the overall number of evidences supporting this Statement
    - `source_counts`: a more detailed breakdown of the `evidence_count` by the specific source from which the evidence was obtained (representing text mining systems, pathway databases, etc.). Detailed source information is available here: https://github.com/sorgerlab/indra#knowledge-sources
    - `belief`: a belief score between 0 and 1 assigned to the statement based on the overall support it has from evidences. Generally, a statement with more evidences from more different sources, and more reliable sources have higher belief. This score can be used for thresholding to retain higher-quality statements. 
    
Let's now make the request to the INDRA Discovery API and look at the relations we got:

In [4]:
import requests

In [5]:
res = requests.post('https://discovery.indra.bio/api/indra_subnetwork_relations',
                    json={'nodes': list(groundings.values())})

In [6]:
subnetwork_relations = res.json()
subnetwork_relations[0]

{'data': {'belief': 0.65,
  'evidence_count': 1,
  'source_counts': '{"reach": 1}',
  'stmt_hash': 19619433452108905,
  'stmt_json': '{"type": "IncreaseAmount", "subj": {"name": "BBC3", "db_refs": {"UP": "Q96PG8", "HGNC": "17868", "TEXT": "PUMA"}}, "obj": {"name": "BCL2L11", "db_refs": {"UP": "O43521", "MESH": "D000072224", "HGNC": "994", "TEXT": "BIM"}}, "belief": 1, "id": "6ac7f599-6367-4482-a42d-81982bec3e30", "matches_hash": "19619433452108905", "evidence": [{"source_api": "reach", "text": "As mitochondria and ER communicate via membrane tethering and a series of signaling events (i.e., induced expression of BIM and PUMA, XREF_FIG) to induce tUPR, we evaluated the expression of Mfn1 and Mfn2, which are implicated in ER-mitochondrial communication.", "annotations": {"found_by": "transcription_1b", "agents": {"coords": [[128, 132], [120, 123]]}}, "epistemics": {"direct": false, "section_type": null}, "text_refs": {"TRID": "20596041", "PMID": "25482509", "PMCID": "PMC4289414", "DOI": 

In [7]:
print('The number of relations we got is %d' % len(subnetwork_relations))

The number of relations we got is 3120


### Examining and filtering the INDRA Statements constituting the sub-network

It is almost always the case that we want to filter INDRA Statements based on some criteria to end up with a more appropriate set for our use case. To do this, we will import INDRA and instantiate the Statement JSONs provided in the relations above for more convenient manipulation.

In [8]:
import json
from indra.statements import stmts_from_json, Complex

stmts = stmts_from_json([json.loads(r['data']['stmt_json']) for r in subnetwork_relations])
stmts

[IncreaseAmount(BBC3(), BCL2L11()),
 DecreaseAmount(BBC3(), BCL2L11()),
 Inhibition(BBC3(), BCL2L11()),
 Activation(BBC3(), BCL2L11()),
 Complex(BBC3(), BCL2L11()),
 Complex(BBC3(), BCL2L1(), BH3 domain()),
 Activation(BBC3(), BCL2L1(muts: (None, None, None))),
 Activation(BBC3(), BCL2L1()),
 Complex(BBC3(), BCL2L1(), sapC()),
 Inhibition(BBC3(bound: [BCL2L1, True]), BCL2L1()),
 Inhibition(BBC3(), BCL2L1()),
 Complex(BBC3(), BCL2L1(), BH3 domains()),
 Complex(BBC3(), BCL2L1(), BCL2L11()),
 Complex(BBC3(), BCL2L1()),
 Activation(BBC3(bound: [PMAIP1, True]), BCL2()),
 IncreaseAmount(BBC3(mods: (modification)), BCL2()),
 IncreaseAmount(BBC3(), BCL2()),
 Complex(BBC3(), BCL2(), BH3 domain()),
 DecreaseAmount(BBC3(), BCL2()),
 Complex(BBC3(), BCL2(), BCL2L1()),
 Complex(BBC3(), BCL2(), BCL2L11()),
 DecreaseAmount(BBC3(mods: (modification)), BCL2()),
 Inhibition(BBC3(), BCL2()),
 Activation(BBC3(), BCL2()),
 Complex(BBC3(), BCL2()),
 Activation(BBC3(bound: [TP53, True]), BAX()),
 Activation(

Looking at the statement above, we see a few issues. For example, INDRA picks up and represent n-ary complexes where n > 2 and these complexes can contain entities not in the original query set (i.e., outside the subnetwork) so it makes sense to filter these out. More generally, statements that are not binary (i.e., contain more than 2 agents like a metabolic conversion reaction or contain just 1 agent like an autophosphorylation) are not that relevant for Boolean modeling so we can filter these out.

In [9]:
stmts = [stmt for stmt in stmts if len(stmt.agent_list()) == 2]
stmts

[IncreaseAmount(BBC3(), BCL2L11()),
 DecreaseAmount(BBC3(), BCL2L11()),
 Inhibition(BBC3(), BCL2L11()),
 Activation(BBC3(), BCL2L11()),
 Complex(BBC3(), BCL2L11()),
 Activation(BBC3(), BCL2L1(muts: (None, None, None))),
 Activation(BBC3(), BCL2L1()),
 Inhibition(BBC3(bound: [BCL2L1, True]), BCL2L1()),
 Inhibition(BBC3(), BCL2L1()),
 Complex(BBC3(), BCL2L1()),
 Activation(BBC3(bound: [PMAIP1, True]), BCL2()),
 IncreaseAmount(BBC3(mods: (modification)), BCL2()),
 IncreaseAmount(BBC3(), BCL2()),
 DecreaseAmount(BBC3(), BCL2()),
 DecreaseAmount(BBC3(mods: (modification)), BCL2()),
 Inhibition(BBC3(), BCL2()),
 Activation(BBC3(), BCL2()),
 Complex(BBC3(), BCL2()),
 Activation(BBC3(bound: [TP53, True]), BAX()),
 Activation(BBC3(bound: [BCL2, True]), BAX()),
 Activation(BBC3(catalytic), BAX(), catalytic),
 DecreaseAmount(BBC3(), BAX()),
 Activation(BBC3(bound: [BCL2L1, True]), BAX()),
 Activation(BBC3(bound: [proteins, True]), BAX()),
 Activation(BBC3(bound: [PMAIP1, True]), BAX()),
 Inhibiti

Next, we notice that there are many statements that only differ from one another because of some detailed agent states (which include modification states, mutations, bound conditions, activities and cellular location). In this use case, keeping track of these statements is not very useful so we can discard them.

In [10]:
stmts = [stmt for stmt in stmts if
  all(
    (not agent.location
     and not agent.mods
     and not agent.mutations
     and not agent.bound_conditions
     and not agent.activity) for agent in stmt.agent_list())]
stmts

[IncreaseAmount(BBC3(), BCL2L11()),
 DecreaseAmount(BBC3(), BCL2L11()),
 Inhibition(BBC3(), BCL2L11()),
 Activation(BBC3(), BCL2L11()),
 Complex(BBC3(), BCL2L11()),
 Activation(BBC3(), BCL2L1()),
 Inhibition(BBC3(), BCL2L1()),
 Complex(BBC3(), BCL2L1()),
 IncreaseAmount(BBC3(), BCL2()),
 DecreaseAmount(BBC3(), BCL2()),
 Inhibition(BBC3(), BCL2()),
 Activation(BBC3(), BCL2()),
 Complex(BBC3(), BCL2()),
 DecreaseAmount(BBC3(), BAX()),
 IncreaseAmount(BBC3(), BAX()),
 Inhibition(BBC3(), BAX()),
 Complex(BBC3(), BAX()),
 Activation(BBC3(), BAX()),
 Activation(BBC3(), BAD()),
 Inhibition(BBC3(), BAD()),
 Complex(BBC3(), BAD()),
 IncreaseAmount(BBC3(), MCL1()),
 Ubiquitination(BBC3(), MCL1()),
 DecreaseAmount(BBC3(), MCL1()),
 Inhibition(BBC3(), MCL1()),
 Activation(BBC3(), MCL1()),
 Complex(BBC3(), MCL1()),
 Activation(BBC3(), FADD()),
 Inhibition(BBC3(), FADD()),
 Complex(BBC3(), CYCS()),
 Inhibition(BBC3(), CYCS()),
 Activation(BBC3(), CYCS()),
 Inhibition(BBC3(), CASP9()),
 Activation(BB

We now have fairly clean binary statements that are well suited to start Boolean modeling. However, we notice that for each pair of agents, there are actually several statement types, sometimes conflicting, being reported (e.g., 
```
IncreaseAmount(BBC3(), BCL2L11()),
DecreaseAmount(BBC3(), BCL2L11())
```
Since Boolean models rely on the polarity of interactions, we can try to resolve these using some heuristic. Let's build up some ad-hoc data structures to explore this issue a bit more. For each pair of agents, we will list the type of interaction and the number/source of evidences supporting it.

In [11]:
# This will be a lookup of evidence source counts by statement hash
source_counts_by_statement_hash = {rel['data']['stmt_hash']: json.loads(rel['data']['source_counts'])
                                   for rel in subnetwork_relations}

In [12]:
def get_stmts_by_agent_pair(stmts, source_counts_by_statement_hash):
    from collections import defaultdict
    stmts_by_agent_pair = defaultdict(list)
    for stmt in stmts:
        # At this point we know we have two agents
        agents = stmt.agent_list()
        stmts_by_agent_pair[(agents[0].name, agents[1].name)].append(
            (stmt, source_counts_by_statement_hash[stmt.get_hash()]))
    # We sort in descending order by evidence count the different statements between each pair of genes
    stmts_by_agent_pair = dict(stmts_by_agent_pair)
    for agent_pair, stmt_tuples in stmts_by_agent_pair.items():
        stmts_by_agent_pair[agent_pair] = sorted(stmt_tuples, key=lambda x: sum(x[1].values()), reverse=True)
    return stmts_by_agent_pair
stmts_by_agent_pair = get_stmts_by_agent_pair(stmts, source_counts_by_statement_hash)
stmts_by_agent_pair

{('BBC3',
  'BCL2L11'): [(Complex(BBC3(), BCL2L11()),
   {'reach': 19,
    'sparser': 43}), (Inhibition(BBC3(), BCL2L11()), {'reach': 8}), (Activation(BBC3(), BCL2L11()),
   {'reach': 7, 'medscan': 1}), (DecreaseAmount(BBC3(), BCL2L11()),
   {'reach': 2}), (IncreaseAmount(BBC3(), BCL2L11()), {'reach': 1})],
 ('BBC3',
  'BCL2L1'): [(Complex(BBC3(), BCL2L1()),
   {'hprd': 3,
    'signor': 1,
    'reach': 61,
    'biogrid': 5,
    'sparser': 49,
    'trips': 2,
    'medscan': 15}), (Inhibition(BBC3(), BCL2L1()),
   {'signor': 1,
    'reach': 8,
    'sparser': 4,
    'medscan': 4}), (Activation(BBC3(), BCL2L1()),
   {'reach': 8, 'medscan': 5})],
 ('BBC3',
  'BCL2'): [(Complex(BBC3(), BCL2()),
   {'hprd': 2,
    'signor': 2,
    'reach': 95,
    'biogrid': 4,
    'sparser': 42,
    'trips': 1,
    'medscan': 4}), (Inhibition(BBC3(), BCL2()),
   {'signor': 2,
    'reach': 39,
    'sparser': 10,
    'medscan': 3}), (Activation(BBC3(), BCL2()),
   {'reach': 28, 'sparser': 1}), (DecreaseAmount(

In [13]:
print('This data structure also allows us to tell that there are %s agent pairs in our statement set which is indicative of the size of the Boolean network we would generate.'
      % len(stmts_by_agent_pair))

This data structure also allows us to tell that there are 401 agent pairs in our statement set which is indicative of the size of the Boolean network we would generate.


Let's look at a specific agent pair example now of BBC3 and BCL2L1:

In [14]:
stmts_by_agent_pair[('BBC3', 'BCL2L1')]

[(Complex(BBC3(), BCL2L1()),
  {'hprd': 3,
   'signor': 1,
   'reach': 61,
   'biogrid': 5,
   'sparser': 49,
   'trips': 2,
   'medscan': 15}),
 (Inhibition(BBC3(), BCL2L1()),
  {'signor': 1, 'reach': 8, 'sparser': 4, 'medscan': 4}),
 (Activation(BBC3(), BCL2L1()), {'reach': 8, 'medscan': 5})]

What we see is the following:
- `Complex`: many evidences including from curated databases and text mining that BBC3 and BCL2L1 bind to form a complex so this is pretty reliably true.
- `Inhibition`: some text mining evidence, and from a curated pathway database (SIGNOR) that BBC3 inhibits BCL2L1.
- `Activation`: interestingly, there is also some evidence that BBC3 activates BCL2L1

How about we look up the evidence supporting the Activation statement? We can do this using the hash of the statement through the following link:

In [15]:
print('http://db.indra.bio/statements/from_hash/%s?format=html' %stmts_by_agent_pair[('BBC3', 'BCL2L1')][2][0].get_hash())

http://db.indra.bio/statements/from_hash/-5075052916332136?format=html


Here we see that these sentences are all misunderstood by the reading systems, and we can actually curate some of the evidences to mark them as incorrect. This will allow us to systematically filter them out. We can now fetch the body of curations collected in the INDRA DB and apply a filter to determine which statements can be thrown away as incorrect.

In [16]:
from indra.sources import indra_db_rest
from indra.tools import assemble_corpus as ac

In [17]:
curations = indra_db_rest.get_curations()

INFO: [2022-03-31 19:53:39] indra.sources.indra_db_rest.util - query: https://db.indra.bio/dev/curation/list
INFO: [2022-03-31 19:53:39] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2022-03-31 19:53:39] indra.sources.indra_db_rest.util - data: None


In [18]:
stmts = ac.filter_by_curation(stmts, curations)

INFO: [2022-03-31 19:53:44] indra.tools.assemble_corpus - Filtering 1408 statements with any incorrect curations...
INFO: [2022-03-31 19:53:44] indra.tools.assemble_corpus - 1400 statements after filter...


As we see above, by browsing the evidences and curating statements, we can systematically eliminate incorrect extractions if necessary.

We can also apply some other heuristics for filtering statements if necessary. We can filter them by for instance the belief score, or more explicit criteria. For example, we could filter out
- any statement that only has evidences from a single text mining system
- is a Complex and only has evidence from Sparser (which tends to produce spurious complexes and if there is no support from other systems these are often incorrect)

In [19]:
from indra.sources import SOURCE_INFO
readers = {k for k, v in SOURCE_INFO.items() if v['type'] == 'reader'}

stmts = [stmt for stmt in stmts
         if not (
             len(source_counts_by_statement_hash[stmt.get_hash()]) == 1
             and set(source_counts_by_statement_hash[stmt.get_hash()]) < readers
            )
        ]
stmts = [stmt for stmt in stmts
         if not (isinstance(stmt, Complex) and
                 set(source_counts_by_statement_hash[stmt.get_hash()]) == {'sparser'}
                )
        ]

In [20]:
print('We are now left with %d statements' % len(stmts))

We are now left with 712 statements


Let's now get back to the issue of conflicting polarities, looking at a second example:

In [21]:
stmts_by_agent_pair = get_stmts_by_agent_pair(stmts, source_counts_by_statement_hash)

In [22]:
stmts_by_agent_pair[('FAS','CASP8')]

[(Activation(FAS(), CASP8()),
  {'eidos': 3, 'reach': 334, 'sparser': 91, 'trips': 3, 'medscan': 35}),
 (Complex(FAS(), CASP8()),
  {'hprd': 2,
   'reach': 37,
   'biogrid': 7,
   'sparser': 37,
   'trips': 2,
   'medscan': 20}),
 (Inhibition(FAS(), CASP8()), {'reach': 29, 'medscan': 7}),
 (IncreaseAmount(FAS(), CASP8()), {'isi': 1, 'reach': 8}),
 (Activation(FAS(), CASP8(), catalytic), {'bel_lc': 4}),
 (DecreaseAmount(FAS(), CASP8()), {'reach': 1, 'medscan': 1})]

In this case we see that Activation has significantly more evidence than the opposite Inhibition.

### Resolving to a consensus polarity
In general, a good heuristic could be to determine polarity of regulation as positive or negative if there is large difference between the evidences for each polarity. Let's attempt to use a simple heuristic to assign a "polarity score" to each agent pair as follows. We consider Activation and IncreaseAmount as full positive regulation, and Inhibition and DecreaseAmount as full negative regulation. Since modifications don't usually have a well determined effect on activity (phosphorylation can be both activating and inhibiting), we take them into account but with a lower weight, say 0.5. We don't score Complexes and other statement types with undetermined polarity.

In [23]:
from indra.statements import *

def polarity_score(stmt_tuples):
    score = 0
    total_evidence = 0
    for stmt, source_counts in stmt_tuples:
        ev_count = sum(source_counts.values())
        if isinstance(stmt, (Activation, IncreaseAmount)):
            score += ev_count
        elif isinstance(stmt, (Inhibition, DecreaseAmount)):
            score -= ev_count
        elif isinstance(stmt, Phosphorylation):
            score += 0.5*ev_count
        elif isinstance(stmt, Dephosphorylation):
            score -= 0.5*ev_count
        total_evidence += ev_count
    return score/total_evidence

In [24]:
polarity_scores = {agent_pair: polarity_score(stmt_tuples)
                   for agent_pair, stmt_tuples in stmts_by_agent_pair.items()}

We can now print the polarity scores in ranked order from "very positive" to "very negative".

In [25]:
for agent_pair, score in sorted(polarity_scores.items(), key=lambda x: x[1], reverse=True):
    print(agent_pair[0], agent_pair[1], score)

BBC3 CASP9 1.0
BBC3 CASP3 1.0
BBC3 BID 1.0
CYCS AKT 1.0
CYCS CASP7 1.0
CYCS FAS 1.0
CYCS apoptosome 1.0
apoptotic process CYCS 1.0
apoptotic process FASLG 1.0
apoptotic process TNF 1.0
FADD CASP9 1.0
FADD TRADD 1.0
FADD TNFRSF1A 1.0
apoptosome CASP9 1.0
apoptosome CASP3 1.0
apoptosome apoptotic process 1.0
MAPK14 BAX 1.0
MCL1 AKT 1.0
BAD CASP9 1.0
BAD CASP3 1.0
BAX BBC3 1.0
BAX CASP8 1.0
BAX CASP7 1.0
BCL2 CASP7 1.0
BCL2L1 MCL1 1.0
BCL2L11 CYCS 1.0
BCL2L11 CASP9 1.0
BCL2L11 FAS 1.0
BCL2L11 BID 1.0
BID CYCS 1.0
BID CASP7 1.0
TNF BCL2L11 1.0
TNF CASP7 1.0
TNF CYCS 1.0
TNF BBC3 1.0
TNFRSF1A MAPK14 1.0
TNFRSF1A CASP3 1.0
FAS BAX 1.0
FAS CYCS 1.0
FAS CASP7 1.0
FASLG BCL2L1 1.0
FASLG CASP9 1.0
FASLG TNFRSF1A 1.0
FASLG TNF 1.0
FASLG BID 1.0
TRADD CASP3 1.0
CASP3 BCL2L11 1.0
CASP3 BAD 1.0
CASP3 FADD 1.0
CASP3 BBC3 1.0
CASP3 TNF 1.0
CASP3 BID 1.0
CASP7 CASP3 1.0
CASP8 CASP7 1.0
CASP8 FASLG 1.0
CASP8 CYCS 1.0
CASP9 BCL2 1.0
CASP9 FASLG 1.0
CASP9 BID 1.0
CASP9 apoptosome 1.0
FASLG CASP3 0.9758064

This score could be the basis of filtering or parameterization for constructing the Boolean model. For simplicity, let's apply the heuristic that we take all polarity scores >= 0 as positive and all < 0 as negative to create the following SIF dump for Boolean modeling.

In [26]:
for agent_pair, score in sorted(polarity_scores.items(), key=lambda x: x[1], reverse=True):
    print('%s,%s,%s' % (agent_pair[0], (1 if score >= 0 else -1), agent_pair[1]))

BBC3,1,CASP9
BBC3,1,CASP3
BBC3,1,BID
CYCS,1,AKT
CYCS,1,CASP7
CYCS,1,FAS
CYCS,1,apoptosome
apoptotic process,1,CYCS
apoptotic process,1,FASLG
apoptotic process,1,TNF
FADD,1,CASP9
FADD,1,TRADD
FADD,1,TNFRSF1A
apoptosome,1,CASP9
apoptosome,1,CASP3
apoptosome,1,apoptotic process
MAPK14,1,BAX
MCL1,1,AKT
BAD,1,CASP9
BAD,1,CASP3
BAX,1,BBC3
BAX,1,CASP8
BAX,1,CASP7
BCL2,1,CASP7
BCL2L1,1,MCL1
BCL2L11,1,CYCS
BCL2L11,1,CASP9
BCL2L11,1,FAS
BCL2L11,1,BID
BID,1,CYCS
BID,1,CASP7
TNF,1,BCL2L11
TNF,1,CASP7
TNF,1,CYCS
TNF,1,BBC3
TNFRSF1A,1,MAPK14
TNFRSF1A,1,CASP3
FAS,1,BAX
FAS,1,CYCS
FAS,1,CASP7
FASLG,1,BCL2L1
FASLG,1,CASP9
FASLG,1,TNFRSF1A
FASLG,1,TNF
FASLG,1,BID
TRADD,1,CASP3
CASP3,1,BCL2L11
CASP3,1,BAD
CASP3,1,FADD
CASP3,1,BBC3
CASP3,1,TNF
CASP3,1,BID
CASP7,1,CASP3
CASP8,1,CASP7
CASP8,1,FASLG
CASP8,1,CYCS
CASP9,1,BCL2
CASP9,1,FASLG
CASP9,1,BID
CASP9,1,apoptosome
FASLG,1,CASP3
CASP9,1,CASP7
TNF,1,MAPK14
TNF,1,CASP8
CASP9,1,CASP3
CYCS,1,CASP9
CASP8,1,CASP3
BID,1,CASP9
CYCS,1,CASP3
CASP8,1,BID
FAS,1,apop

The above could be the prior network structure for a Boolean model.